In [1]:
# 경고 메시지가 안나오게..
import warnings
warnings.filterwarnings('ignore')

# 기본
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차 검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# 저장
import pickle

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# 랜덤 시드값을 설정한다.
# 텐서플로에서 최초의 가중치와 바이어스를 랜덤하게 설정해서 시작한다.
# 랜덤 시드를 고정하면 예제를 실행할 때 마다 같은 패턴의 랜덤으로
# 설정할 수 있다. 앞으로 테스트해 보는 예제에서 모든 조건을 동일하게
# 맞추고 테스트를 해야 정확한 테스트를 할 수 있기 때문에
# 가중치와 바이어스의 최초값을 항상 같은 값을 갖게 하도록 랜덤시드를
# 설정했다.
np.random.seed(3)
tf.random.set_seed(3)

In [3]:
# 데이터를 읽어온다.
df1 = pd.read_csv('data/ThoraricSurgery.csv')
df1.head()

,293,1,3.8,2.8,0,0.1,0.2,0.3,0.4,0.5,12,0.6,0.7,0.8,1.1,0.9,62,0.10
0,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
1,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
2,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
3,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
4,18,2,2.96,1.67,0,0,0,0,0,0,12,0,0,0,1,0,61,0


In [4]:
# 입력과 결과로 나눈다.
X = df1.drop('0.10', axis=1)
Y = df1['0.10']

display(X)
display(Y)

,293,1,3.8,2.8,0,0.1,0.2,0.3,0.4,0.5,12,0.6,0.7,0.8,1.1,0.9,62
0,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60
1,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66
2,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80
3,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56
4,18,2,2.96,1.67,0,0,0,0,0,0,12,0,0,0,1,0,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76
465,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77
466,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62
467,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58


0      0
1      1
2      1
3      0
4      0
      ..
464    0
465    0
466    0
467    1
468    0
Name: 0.10, Length: 469, dtype: int64

In [5]:
# 딥러닝 구조를 설정한다.
# 각 층의 순서를 관리하는 객체를 생성
model = Sequential()
# 층을 쌓는 부분
# 첫 번째 층은 입력층과 은닉층이 같이 생긴다.
# 마지막 층은 출력층에 해당한다.
# 그 외의 모든 층들은 모두 은닉층에 해당한다.
# 노드가 많아지면 계산이 많아지기 때문에 보다 정확해질 수도 있다.
# 과적합의 우려가 있고 시간이 오래 걸린다.
# 은닉층이 많이지면 계산이 많아지기 때문에 보다 정확해질 수도 있다.
# 과적합의 우려가 있고 시간이 오래 걸린다.

# 층에 대한 설정
# 첫번째 숫자 : 층 내부의 노드의 개수. 마지막에 설정하는 출력층의 경우 예측하는
# 결과의 개수에 맞춰준다.

# activation : 활성화 함수. 은닉층들은 가중치 소실을 예방하기 위해 relu로
# 설정한다. 출력층은 출력 결과에 따라 그에 맞는 활성화 함수를 사용한다.

# input_dim : 첫번째 설정하는 은닉층은 입력층과 같이 생성된다.
# 이 때 생성되는 입력층의 노드의 개수를 설정하는 곳이다. 노드의 개수는
# 학습을 위한 입력데이터의 컬럼 개수와 동일하게 설정한다.

model.add(Dense(30, input_dim= 17, activation='relu'))
model.add(Dense(1, activation= 'sigmoid'))

In [6]:
# 딥러닝 구조를 컴파일한다.
# 위에서 설정한 각 층의 정보를 토대로 딥러닝 모델을 생성한다.
# loss : 손실함수, 예측결과와 진짜 결과의 오차 정도를 계산하는 함수
# metrics : 손실함수를 통해 구한 오차 정도를 평가할 지표를 설정한다.
# optimizer : 손실 정도에 따라 가중치를 수정하는 경사하강법 설정

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [7]:
# 학습시작

# X : 입력
# Y : 결과
# epochs : 학습과 오차수정(역전차) 작업을 얼마나 시도할 것인지..
# 최적의 값으로 수정했는데도 계속 작업을 해나가면 오히려 성능이 떨어지는 경우가
# 있을 수 있다. 이 때문에 적당한 선에서 끊어줘야 한다.
# batch_size : 한번에 몇개의 로우를 추출해서 학습시킬것인지.. 메모리가 허락
# 하는한 최대의 숫자를 잡아주면 되지만 적게 주더라도 크게 차이가 나지는 않는다.

model.fit(X, Y, epochs=100, batch_size=10)

Epoch 1/100
47/47 [==============================] - 1s 2ms/step - loss: 0.1497 - accuracy: 0.8328
Epoch 2/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1458 - accuracy: 0.8520
Epoch 3/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1690 - accuracy: 0.8262
Epoch 4/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1465 - accuracy: 0.8370
Epoch 5/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1423 - accuracy: 0.8474
Epoch 6/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1441 - accuracy: 0.8407
Epoch 7/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1558 - accuracy: 0.8404
Epoch 8/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1527 - accuracy: 0.8453
Epoch 9/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1413 - accuracy: 0.8499
Epoch 10/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1175 - accuracy: 0.8698
Epoch 11/

47/47 [==============================] - 0s 2ms/step - loss: 0.1494 - accuracy: 0.8447
Epoch 84/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1557 - accuracy: 0.8341
Epoch 85/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1306 - accuracy: 0.8436
Epoch 86/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1195 - accuracy: 0.8452
Epoch 87/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1347 - accuracy: 0.8504
Epoch 88/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1254 - accuracy: 0.8497
Epoch 89/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1269 - accuracy: 0.8500
Epoch 90/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1207 - accuracy: 0.8596
Epoch 91/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1356 - accuracy: 0.8586
Epoch 92/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1305 - accuracy: 0.8556
Epoch 93/100


In [9]:
r1 = model.evaluate(X, Y)
print(f'손실률 : {r1[0]}')
print(f'정확도 : {r1[1]}')


15/15 [==============================] - 0s 1ms/step - loss: 0.1176 - accuracy: 0.8571
손실률 : 0.11758221685886383
정확도 : 0.8571428656578064


In [11]:
# result = model.predict_classes(X)
# result

result = (model.predict(X) > 0.5).astype("int32")
result


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [12]:
# 정확도 확인
r1 = accuracy_score(Y, result)
r1

0.8571428571428571